In [1]:
## import the libraries
from binance.client import Client
import pandas as pd
import requests

In [2]:
## connect with python
api = '33ced6236b5dcc64077883472fd37a3aae566b82c13008468d8239bae0bd158d'
api_secret = 'e053e6ff070a9f4b6881457848b47bac4e1c03b41b9ad2f8f2fa9a7b25e81559'

client = Client(api, api_secret, tld="com", testnet=True)

In [3]:
## fetch current account balance
def get_balance():
    x = client.futures_account()
    df = pd.DataFrame(x['assets'])
    print(df)
    
get_balance()

  asset   walletBalance unrealizedProfit   marginBalance maintMargin  \
0   ETH      0.00000000       0.00000000      0.00000000  0.00000000   
1   BTC      0.00000000       0.00000000      0.00000000  0.00000000   
2   BNB      0.00000000       0.00000000      0.00000000  0.00000000   
3  USDT  12875.89664350       0.00000000  12875.89664350  0.00000000   
4  USDC      0.00000000       0.00000000      0.00000000  0.00000000   
5  BUSD      0.00000000       0.00000000      0.00000000  0.00000000   

  initialMargin positionInitialMargin openOrderInitialMargin  \
0    0.00000000            0.00000000             0.00000000   
1    0.00000000            0.00000000             0.00000000   
2    0.00000000            0.00000000             0.00000000   
3    0.00000000            0.00000000             0.00000000   
4    0.00000000            0.00000000             0.00000000   
5    0.00000000            0.00000000             0.00000000   

  maxWithdrawAmount crossWalletBalance  crossU

## Placing Buy Limit and Sell Limit Orders

In [4]:
## placing buy limit order
def buy_limit(symbol,quantity,price):
    output = client.futures_create_order(
        symbol=symbol,
        side=Client.SIDE_BUY,
        type=Client.FUTURE_ORDER_TYPE_LIMIT,
        timeInForce=Client.TIME_IN_FORCE_GTC,
        quantity=quantity,
        price=price,
    )
    print(output)

In [5]:
## placing sell limit order
def sell_limit(symbol,quantity,price):
    output = client.futures_create_order(
        symbol=symbol,
        side=Client.SIDE_SELL,
        type=Client.FUTURE_ORDER_TYPE_LIMIT,
        timeInForce=Client.TIME_IN_FORCE_GTC,
        quantity=quantity,
        price=price,
    )
    print(output)

In [6]:
## placing buy limit order
buy_limit("BTCUSDT", 0.02, 43000)

{'orderId': 3604452268, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': '478uRZeqYJsMUeFsyze9T0', 'price': '43000.00', 'avgPrice': '0.00', 'origQty': '0.020', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': 0, 'updateTime': 1704126825404}


In [7]:
## placing sell limit order
sell_limit("BTCUSDT", 0.02, 44000)

{'orderId': 3604452326, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'RP5chEvlekxLqoBuoOmZfu', 'price': '44000.00', 'avgPrice': '0.00', 'origQty': '0.020', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': 0, 'updateTime': 1704126825858}


## Closing Open Orders 

In [8]:
## closing all open orders
def close_orders(symbol):
    x = client.futures_get_open_orders(symbol=symbol)
    df = pd.DataFrame(x)
    for index in df.index:
        client.futures_cancel_order(symbol=symbol,orderId=df["orderId"][index])
        
close_orders("BTCUSDT")

In [9]:
## closing buy orders
def close_buy_orders(symbol):
    x = client.futures_get_open_orders(symbol=symbol)
    df = pd.DataFrame(x)
    df = df[df["side"]=="BUY"]
    for index in df.index:
        client.futures_cancel_order(symbol=symbol,orderId=df["orderId"][index])

In [10]:
## closing sell orders
def close_sell_orders(symbol):
    x = client.futures_get_open_orders(symbol=symbol)
    df = pd.DataFrame(x)
    df = df[df["side"]=="SELL"]
    for index in df.index:
        client.futures_cancel_order(symbol=symbol,orderId=df["orderId"][index])

## Get the Direction of Running Positions

In [11]:
## getting direction of running positions
def get_direction(symbol):
    x = client.futures_position_information(symbol=symbol)
    df = pd.DataFrame(x)
    if float(df["positionAmt"].sum()) > 0:
        return "LONG"
    if float(df["positionAmt"].sum()) < 0:
        return "SHORT"
    else:
        return "FLAT"

## Live Streaming Current Market Price

In [12]:
## streaming price
def get_current_price(symbol):
    response = requests.get(f'https://testnet.binancefuture.com/fapi/v1/ticker/price?symbol={symbol}')
    price = float(response.json()['price'])
    return price

In [ ]:
try:  
    while True:
        price = get_current_price("BTCUSDT")
        print(price)
except KeyboardInterrupt:
    pass

## Drawing the Grid with Python

In [14]:
symbol = "BTCUSDT"
no_of_decimals = 1
volume = 0.01
proportion = 0.04
n = 10

In [15]:
## getting the market price
def get_mark_price(symbol):
    x = client.get_symbol_ticker(symbol=symbol)
    price = float(x["price"])
    return price

In [16]:
## placing orders like a grid
def draw_grid(n):
    pct_change = 1
    adj_sell = 1.2
    current_price = get_mark_price(symbol)
    for i in range(n):
        sell_price = float(round(((pct_change/100)*current_price*adj_sell*proportion) + current_price, no_of_decimals))
        sell_limit(symbol,volume,sell_price)
        pct_change += 1
        adj_sell += 0.2

    pct_change = -1
    adj_buy = 1.2
    current_price = get_mark_price(symbol)
    for i in range(n):
        buy_price = float(round(((pct_change/100)*current_price*adj_sell*proportion) + current_price, no_of_decimals))
        buy_limit(symbol,volume,buy_price)
        pct_change -= 1
        adj_buy += 0.2
        
draw_grid(n)

{'orderId': 3604452738, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'b4n0kLEQbC2TIgamjeQB3o', 'price': '42738.60', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': 0, 'updateTime': 1704126833660}
{'orderId': 3604452792, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': '2fTVXUELgU5mPXMuMK0UdQ', 'price': '42765.90', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origTyp

{'orderId': 3604453051, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'YZcjmaHwiY6M8hIVJsrbpK', 'price': '42335.30', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': 0, 'updateTime': 1704126840189}
{'orderId': 3604453072, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'z4yq0GzDRGrBajoSHvOLFe', 'price': '42280.60', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType'

## Calculating the Take Profit Level

In [17]:
## calculate take profit price
def cal_tp_level(symbol,tp):
    try:
        x = client.futures_position_information(symbol=symbol)
        df = pd.DataFrame(x)
        df = df.loc[df["positionAmt"] != "0.000"]
        t_margin = (float(df["entryPrice"][0]) * abs(float(df["positionAmt"][0]))) / float(df["leverage"][0])
        profit = float(t_margin*tp*0.01)
        price = round((profit / float(df["positionAmt"][0])) + float(df["entryPrice"][0]), no_of_decimals)
        t_position_amt = 0
        for index in df.index:
            t_position_amt += abs(float(df["positionAmt"][index]))
        return price,t_position_amt

    except:
        pass

## Placing the Take Profit Order

In [18]:
## placing take profit order
def place_tp_order(symbol,price,t_position_amt,direction):
    try:
        if direction == "LONG":
            sell_limit(symbol,t_position_amt,price)
        if direction == "SHORT":
            buy_limit(symbol, t_position_amt, price)
    except:
        place_tp_order(symbol, price, t_position_amt, direction)